In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import json
import time
from io import StringIO
from datetime import datetime # for report
import re

from sklearn.linear_model import LinearRegression
from scipy import integrate

# import utility functions
from my_utils import *


In [2]:
template_path = "./data/pro_template.csv"
path_samples = "./data/pro_test_samples/"
fn_to_exclude_ls = [] # list of files to ignore within path_samples. Leave empty if everything is to be included.
#manual_calc_path = "/Users/dteng/Documents/zdata/20220111_DEN_Proline_CombinedData.csv"
lr_results_path = "./results/lr-results-sample.json"

normxcorr_th = 0.0 # set to this number to filter out multiplets which aren't at least normxcorr_th, i.e. poor fits
ref_pk_window = [-0.02, 0.02]
ref_pk_tolerance_window = [0,0]
multiplets_ls = [[1.9,2.15], 
                 [2.305, 2.408],
                 [3.25, 3.5],
                 [4.1, 4.2]]
search_region_padding_size = 0.02
dept_var = "auc_target"
conc_dict = {
      "01":5104,
      "02":2041.6,
      "03":1020.8,
      "04":816.64,
      "05":510.4,
      "06":306.24,
      "07":127.6,
      "08":51.04,
      "09":25.52,
      "10":10.208,
      "11":0
   }

fn_out_plot = "./results/pred_results.html"
fn_out_df = "./results/pred_results.csv"


In [3]:
# ========== load sample data ==========
df_dict = {}
for fn in os.listdir(path_samples):
    if (".csv" in fn) and (fn not in fn_to_exclude_ls):
        dt = pd.read_csv(path_samples+"/"+fn)
        k = fn.replace(".csv", "")

        # adjust ref pk
        dt = adjust_to_ref_peak(dt, ref_pk_window, ref_pk_tolerance_window)
        df_dict[k] = dt
        
# load STD template(s)
template_df = pd.read_csv(template_path)

# load fitted gradient from lr_fitting
with open(lr_results_path) as f:
    lr_results_dict = json.load(f)

# ========== run 1d_std_search ==========
results_dict = {}
for k in sorted(list(df_dict.keys())):
    target_df = df_dict[k]
    results_dict[k] = do_1d_std_search(query_df=template_df, 
                                       target_df=target_df,
                                       multiplets_ls=multiplets_ls,
                                       search_region_padding_size=search_region_padding_size
                                      )


In [4]:
t0 = time.perf_counter()

# ========== get df_auc ==========
df_conc = get_df_auc(template_df, df_dict, results_dict, multiplets_ls)

df_conc["conc_pred"] = df_conc[dept_var].values * lr_results_dict["multiplet_1"]["gradient"]
df_conc["conc_pred_lower"] = df_conc[dept_var].values * lr_results_dict["multiplet_1"]["gradient_ci"][0]
df_conc["conc_pred_upper"] = df_conc[dept_var].values * lr_results_dict["multiplet_1"]["gradient_ci"][1]

# some final modifications
dt2 = df_conc.loc[(df_conc["multiplet"]=="multiplet_1") & (df_conc["normxcorr"]>normxcorr_th)]
#dz = pd.merge(d_manual, dt2, on="sample_name", how="right")
#temp_arr = dz["manual_conc"] - dz["conc_pred"]
#dz["conc_error_manual-pred"] = temp_arr
#temp_arr = ((dz["manual_conc"].values - dz["conc_pred"].values)/dz["manual_conc"].values)*100
#dz["pct_error"] = temp_arr

print("Done in %.2fs" % (time.perf_counter() - t0))


Done in 0.39s


In [5]:
# viz plot for visual confirmation/QC
t0 = time.time()
keys_ls = sorted(list(df_dict.keys()))
fig, ax = plt.subplots(nrows=len(keys_ls), # top row for LR results
                       ncols=len(multiplets_ls), 
                       figsize=(30, (len(keys_ls)+1)*4)
                      )

# viz fit
for i in range(len(keys_ls)):
    k = keys_ls[i]
    
    for j in range(len(multiplets_ls)):
        mcoords = multiplets_ls[j]
        # plot sample (blue)
        dt_target = df_dict[k]
        dt_target = dt_target.loc[(dt_target["ppm"]>min(mcoords)-search_region_padding_size) & 
                                  (dt_target["ppm"]<max(mcoords)+search_region_padding_size)].copy()
        target_intensity_arr = dt_target["intensity"].values - min(dt_target["intensity"].values)
        dt_target["intensity"] = target_intensity_arr

        ax[i, j].plot(dt_target.ppm.values, dt_target.intensity.values, c="steelblue")
        
        # plot fit (red)
        # shift ppm
        ppm_shift = max(mcoords) - max(results_dict[k][f"multiplet_{j}"]["multiplet_match_ppm"][0])
        multiplet_df = template_df.loc[(template_df["ppm"]>min(mcoords)) & (template_df["ppm"]<max(mcoords))].copy()
        new_ppm_arr = multiplet_df.ppm.values - ppm_shift
        multiplet_df["ppm"] = new_ppm_arr

        # floor spectra
        query_intensity_arr = multiplet_df["intensity"].values - min(multiplet_df["intensity"].values)
        multiplet_df["intensity"] = query_intensity_arr

        # get sf
        sf = df_conc.loc[(df_conc["multiplet"]==f"multiplet_{j}") & (df_conc["sample_name"]==k)].scaling_factor.values[0]

        ax[i, j].plot(multiplet_df.ppm.values, multiplet_df.intensity.values/sf, c="indianred")

        # set bg colour
        xcorr = max(results_dict[k][f"multiplet_{j}"]["rho_ls"])
        bg_colour = "#FE9FA5"
        if xcorr > 0.8 and xcorr < 0.95:
            bg_colour = "#FEDA96"
        elif xcorr >= 0.95:
            bg_colour = "#96FEBF"
        ax[i, j].set_facecolor(bg_colour)
        ax[i, j].set_title(f"norm_xcorr={round(xcorr, 3)}", fontsize=25)
        plt.setp(ax[i, j].get_xticklabels(), fontsize=20)
        plt.setp(ax[i, j].get_yticklabels(), fontsize=20)
        
        # yaxis labels with sample names
        if j == 0:
            ax[i, j].set_ylabel(k.replace("20220111_", "").replace("20220113_", ""), fontsize=21)

plt.subplots_adjust(hspace=0, wspace=0)
plt.tight_layout()
plt.close()

print("Done in %.2fs" % (time.time() - t0))

i = StringIO()
fig.savefig(i, format="svg")
output_svg = i.getvalue().strip().split("\n")


Done in 8.44s


In [6]:
# resize svg
svg_ls = resize_svg(output_svg, resize_coeff=0.5)

# prep html form
html_contents = ["<html><head></head><body>"]

html_contents.append(f"<li>Report generated: {datetime.today().strftime('%Y-%m-%d')}</li>")
html_contents.append(f"<li>num_samples = {len(keys_ls)}</li>")
html_contents.append(f"<li>normxcorr threshold = {normxcorr_th}</li>")
html_contents.append(f"<li>ref peak window = {ref_pk_window}</li>")
html_contents.append(f"<li>ref peak tolerance window = {ref_pk_tolerance_window}</li>")
html_contents.append(f"<li>AUC variable = {dept_var}</li>")
html_contents.append(f"<li>search region padding size (ppm) = {search_region_padding_size}</li>")
html_contents.append(f"<li>Output csv = {fn_out_df}</li>")

html_contents.append(f"<h3>LR Params</h3>")
html_contents.append(f"<li>Intercept = {lr_results_dict['multiplet_1']['intercept']}(ignore this, reported for completeness)</li>")
html_contents.append(f"<li>Gradient = {lr_results_dict['multiplet_1']['gradient']}, CI = {lr_results_dict['multiplet_1']['gradient_ci']}</li>")
html_contents.append(f"<li>R^2 = {lr_results_dict['multiplet_1']['rsquared']}</li>")
html_contents.append(f"<li>Adjusted R^2 = {lr_results_dict['multiplet_1']['rsquared_adj']}</li>")
html_contents.append("<hr>")
for line in output_svg:
    html_contents.append(line)
html_contents.append("</body></html>")

# write out
with open(fn_out_plot, "w") as f:
    for line in html_contents:
        f.write(line)

dt2.to_csv(fn_out_df, index=False)

print(f"Wrote out to:\n{fn_out_plot}\n{fn_out_df}")

Wrote out to:
./results/pred_results.html
./results/pred_results.csv
